In [ ]:
import sys
sys.path.append('/home/ABTLUS/victor.secco/data_processing/') 
import os
import numpy as np
import matplotlib.pyplot as plt
import pyFAI
from mypackages.edp_processing import ImageAnalysis, ImageProcessing, peak_calibration
import hyperspy.api as hs

In [ ]:
#Load .ser data
directory = '/home/ABTLUS/victor.secco/data_processing/ED/Fe3O4/Maghemite/Fe2O3_OAm/ROIS'
names = [x for x in os.listdir(directory) if x.endswith('.ser')] 

file_path = os.path.join(directory, names[2])
data = hs.load(file_path, signal_type='diffraction', lazy=True)
imgs = np.array(data)
img= imgs[5]

In [ ]:
del data 
del imgs

In [ ]:
analysis = ImageAnalysis()
#The outputs are all 
center_x, center_y, r  = analysis.find_center(img, r = 1, #smallest radius
                                                            R = 5000, #biggest radius
                                                            threshold = 100, #threshold applied to the difratogram
                                                            edges_thresh1 = 255, #canny edge detector threshold
                                                            edges_thresh2 = 10)  #canny edge detector threshold


square_size = 512  # Adjust to your desired size

# Ensure the square stays within bounds
top = max(0, center_y - square_size)  # Top boundary
bottom = center_y  # Bottom boundary is the center_y coordinate
left = max(0, center_x - square_size // 2)  # Left boundary
right = min(4096, center_x + square_size // 2)  # Right boundary

# Slice the square region above the point
sliced_square = img[top:bottom, left:right]

line_x = sliced_square.shape[1] // 2  # Midpoint of the height

# Extract the line profile (horizontal line)
line_profile = sliced_square[:, line_x]

plt.figure(figsize=(8, 8))
plt.imshow(sliced_square, cmap='gray', origin='upper')
plt.axvline(x=line_x, color='red', linestyle='--', label=f"Line Profile at y={line_y}")
plt.colorbar(label='Intensity')
plt.title('Sliced Square Image with Line Overlay')
plt.xlabel('X Pixels')
plt.ylabel('Y Pixels')
plt.legend()
plt.show()


# Plot the line profile
plt.figure(figsize=(8, 4))
plt.plot(line_profile, label=f"Line Profile at y={line_x}", color='red')
plt.xlabel('Pixel Index')
plt.ylabel('Intensity')
plt.title('Line Profile')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
start_x, end_x = 412, 512

# Slice the line profile for the interval
line_profile_interval = line_profile[start_x:end_x]

# Differentiate the sliced line profile
differentiated_profile = np.gradient(line_profile_interval)

# Plot the sliced profile and its derivative
plt.figure(figsize=(8, 4))
plt.plot(range(start_x, end_x), abs(differentiated_profile), label="Differentiated Profile", linestyle='--')
plt.xlabel('Pixel Index')
plt.ylabel('Intensity / Gradient')
plt.title('Line Profile and Its Derivative (x = 412 to 512)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Define zero-padding length
original_fourier_transform = np.fft.fft(differentiated_profile)
original_frequencies = np.fft.fftfreq(len(differentiated_profile))
original_amplitudes = np.abs(original_fourier_transform[:len(original_frequencies) // 2])
original_amplitudes_normalized = original_amplitudes / np.max(original_amplitudes)
original_positive_frequencies = original_frequencies[:len(original_frequencies) // 2]

# Zero-padding to length 512
padded_length = 512
padded_profile = np.pad(differentiated_profile, (0, padded_length - len(differentiated_profile)), 'constant')
padded_fourier_transform = np.fft.fft(padded_profile)
padded_frequencies = np.fft.fftfreq(len(padded_profile))
padded_amplitudes = np.abs(padded_fourier_transform[:len(padded_frequencies) // 2])
padded_amplitudes_normalized = padded_amplitudes / np.max(padded_amplitudes)
padded_positive_frequencies = padded_frequencies[:len(padded_frequencies) // 2]

# Plot for comparison
plt.figure(figsize=(10, 6))
plt.plot(original_positive_frequencies, original_amplitudes_normalized, label="Original (Unpadded)", linestyle="--")
plt.plot(padded_positive_frequencies, padded_amplitudes_normalized, label="512-Padded", linestyle="-")
plt.xlabel('Frequency (arbitrary units)')
plt.ylabel('Normalized Amplitude')
plt.title('Comparison of Original and 512-Padded FFT Profiles')
plt.legend()
plt.grid(True)
plt.show()
